In [1]:
import os
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import Graph, StateGraph
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.messages import HumanMessage
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import json
import csv
from dotenv import load_dotenv
from langgraph.graph import StateGraph, END

# Load environment variables from .env file
load_dotenv()

# Initialize Google Generative AI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=os.getenv("GEMINI_API_KEY"))

# Initialize Tavily Search
tavily_tool = TavilySearchResults(api_key=os.getenv("TAVILY_API_KEY"))

In [ ]:
# Create a vector store
def create_vector_store():
    """
    Create a vector store from PDF documents.
    
    This function loads PDF documents, splits the text into chunks, and creates embeddings using Google Generative AI.
    
    Returns:
        FAISS: A vector store containing the text and corresponding embeddings.
    """
    
    try:
        # Try to load the existing vector store first
        embeddings = GoogleGenerativeAIEmbeddings(
            model="models/gemini-embedding-exp-03-07", 
            google_api_key=os.getenv("GEMINI_API_KEY")
        )
        vector_store = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)
        print("Loaded existing vector store.")
        return vector_store
    except Exception as e:
        print(f"Could not load existing vector store: {e}. Creating new vector store...")
    
    # Loading documents
    documents = []
    try:
        for i in range(1, 4):
            try:
                loader = PyPDFLoader(f"malraia_1.pdf")
                documents.extend(loader.load())
            except FileNotFoundError:
                print(f"File malraia_{i}.pdf not found, skipping.")
    except Exception as e:
        print(f"Error loading documents: {e}")
        # Fallback to loading just the first document
        try:
            loader = PyPDFLoader("malraia_1.pdf")
            documents.extend(loader.load())
        except FileNotFoundError:
            print("Critical: No documents found")
            return None
    
    # Splitting documents and creating text representations
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    texts = text_splitter.split_documents(documents)
    print(f"Number of text chunks: {len(texts)}")
    
    # Create embeddings using a Google Generative AI model
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/gemini-embedding-exp-03-07", 
        google_api_key=os.getenv("GEMINI_API_KEY")
    )
    
    # Create a vector store using FAISS from the provided text chunks and embeddings
    vector_store = FAISS.from_documents(texts, embedding=embeddings)
    
    # Save the vector store locally with the name "faiss_index"
    vector_store.save_local("faiss_index")
    return vector_store


# Initialize vector store
try:
    vector_store = create_vector_store()
    if vector_store is None:
        raise ValueError("Failed to create vector store")
except Exception as e:
    print(f"Error creating vector store: {e}")
    # # Create a simple dummy vector store for testing
    # vector_store = FAISS.from_texts(["Dummy text about malaria for testing"], 
    #                                embedding=GoogleGenerativeAIEmbeddings(
    #                                    model="models/gemini-embedding-exp-03-07",
    #                                    google_api_key=os.getenv("GEMINI_API_KEY")
    #                                ))


Loaded existing vector store.


In [ ]:
# Create a search function for the vector store
def search_vector_store(query: str) -> str:
    try:
        print(f"Starting vector search for query: {query}")
        results = vector_store.similarity_search(query, k=1)
        print(f"Vector search results: {results}")
        return results[0].page_content
    except Exception as e:
        print(f"Error searching vector store: {e}")
        return "No information found in the vector store."
    
    
# Create a Tavily search function   
def execute_tavily_tool(query: str):
    try:
        print(f"Starting Tavily search for query: {query}")
        results = tavily_tool.run(query)
        print(f"Tavily search results: {results}")
        return results
    except Exception as e:
        print(f"Error in Tavily search: {e}")
        return "No information found using Tavily."
    
# Create a tool for the vector store
vector_search_tool = Tool(
    name="VectorSearch",
    func=search_vector_store,
    description="Searches the vector store for relevant information about diseases."
)

# Create a tool for Tavily search
tavily_web_search_tool =  Tool(
        name="TavilySearch",
        func=execute_tavily_tool,
        description="Searches the web for relevant information about diseases."
    )

# Create an agent
tools = [vector_search_tool, tavily_tool]


In [4]:
search_vector_store("malaria")

Starting vector search for query: malaria
Vector search results: [Document(metadata={'source': 'malraia_1.pdf', 'page': 1}, page_content='WHO guidelines for malaria - 30 November 2024 This document is a PDF generated from the WHO guidelines for malaria hosted on the MAGICapp online \nplatform: https://app.magicapp.org/#/guideline/LwRMXj. Each time the content of the platform is updated, a new PDF version of the \nGuidelines will be downloadable on the WHO Global Malaria Programme website to facilitate access where the Internet is not \navailable. Users should note the downloaded PDFs of the Guidelines may be outdated and not contain the latest recommendations. \nPlease consult with the website for the most up-to-date version of the Guidelines (https://www.who.int/teams/global-malaria-\nprogramme). \nContact \nWHO Global Malaria Programme \nAppia Avenue 20, 1202 Geneva, Switzerland \ngmpfeedback@who.int \nhttps://www.who.int/teams/global-malaria-programme \nSponsors/Funding \nFunding fo

'WHO guidelines for malaria - 30 November 2024 This document is a PDF generated from the WHO guidelines for malaria hosted on the MAGICapp online \nplatform: https://app.magicapp.org/#/guideline/LwRMXj. Each time the content of the platform is updated, a new PDF version of the \nGuidelines will be downloadable on the WHO Global Malaria Programme website to facilitate access where the Internet is not \navailable. Users should note the downloaded PDFs of the Guidelines may be outdated and not contain the latest recommendations. \nPlease consult with the website for the most up-to-date version of the Guidelines (https://www.who.int/teams/global-malaria-\nprogramme). \nContact \nWHO Global Malaria Programme \nAppia Avenue 20, 1202 Geneva, Switzerland \ngmpfeedback@who.int \nhttps://www.who.int/teams/global-malaria-programme \nSponsors/Funding \nFunding for the development and publication of the Guidelines was gratefully received from the Bill & Melinda Gates Foundation, The \nGlobal Fund, 

In [5]:
execute_tavily_tool("ICD code malaria")

Starting Tavily search for query: ICD code malaria
Tavily search results: [{'url': 'https://www.aapc.com/codes/icd-10-codes/B51?srsltid=AfmBOoq5v4jiP_vXn7oIqUEkTR8p-7XnLePXxfB2hUhrZu6XBFs-Ih6e', 'content': 'ICD-10 code B51 for Plasmodium vivax malaria is a medical classification as listed by WHO under the range -Protozoal diseases .'}, {'url': 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/B50-B64/B54-/B54', 'content': 'malarial B54 - see also Malaria [...] 2025\n\n2025 ICD-10-CM Diagnosis Code B54\n\nUnspecified malaria [...] Anemia (essential) (general) (hemoglobin deficiency) (infantile) (primary) (profound) D64.9ICD-10-CM Diagnosis Code D64.9Anemia, unspecified2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 Billable/Specific Code  malarial B54 - see also Malaria marsh B54 - see also Malaria paludal B54 - see also Malaria Chill(s) R68.83ICD-10-CM Diagnosis Code R68.83Chills (without fever)2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 Billable/Specific Code Applicable ToChill

[{'url': 'https://www.aapc.com/codes/icd-10-codes/B51?srsltid=AfmBOoq5v4jiP_vXn7oIqUEkTR8p-7XnLePXxfB2hUhrZu6XBFs-Ih6e',
  'content': 'ICD-10 code B51 for Plasmodium vivax malaria is a medical classification as listed by WHO under the range -Protozoal diseases .'},
 {'url': 'https://www.icd10data.com/ICD10CM/Codes/A00-B99/B50-B64/B54-/B54',
  'content': 'malarial B54 - see also Malaria [...] 2025\n\n2025 ICD-10-CM Diagnosis Code B54\n\nUnspecified malaria [...] Anemia (essential) (general) (hemoglobin deficiency) (infantile) (primary) (profound) D64.9ICD-10-CM Diagnosis Code D64.9Anemia, unspecified2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 Billable/Specific Code  malarial B54 - see also Malaria marsh B54 - see also Malaria paludal B54 - see also Malaria Chill(s) R68.83ICD-10-CM Diagnosis Code R68.83Chills (without fever)2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 Billable/Specific Code Applicable ToChills NOSType 1 Excludeschills with'},
 {'url': 'https://health.mil/Refer

In [ ]:
# Define the agent's prompt
prompt = PromptTemplate.from_template(
    """You are a medical information retrieval agent. Your task is to find information about diseases and their ICD codes.

    Tools you can use:
    {tools}

    Tool name: {tool_names}

    Human: {human_input}
    You must respond in the following format:
        Thought: <Your thought process>
        Action: <The action you will take>
        Action Input: <The input for the action>

    Always use the following steps:
    1. Use VectorSearch to retrieve the disease description.
    2. Use TavilySearch to retrieve the ICD codes for the disease.
    3. Compile the results and return them in this JSON format:
    {{
        "disease": "<disease_name>",
        "description": "<description>",
        "icd_codes": ["<code1>", "<code2>", ...]
    }}
    4. Once you have compiled the JSON response, stop and return the result. Do not take any further actions.

    {agent_scratchpad}
    """
)

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
        agent=agent, 
        tools=tools, 
        handle_parsing_errors=True,
        max_iterations=2,
        verbose=True,
        return_intermediate_steps=True,
        max_execution_time=120.0)

In [7]:
# Define the state
class AgentState(TypedDict):
    human_input: str
    messages: Sequence[HumanMessage]
    results: dict

# Define the nodes
def agent_node(state: AgentState):
    """Process the agent's response"""
    try:
        # Execute the agent
        result = agent_executor.invoke({
            "human_input": state["human_input"]
        })
        
        print("Raw result from agent:", result)
        
        # Check if the result contains the final JSON output
        if isinstance(result, dict) and "disease" in result and "description" in result and "icd_codes" in result:
            print("Final result obtained. Terminating chain.")
            return {
                "messages": state.get("messages", []),  # Keep existing messages
                "results": result  # Return the final result
            }
        
        # Continue processing if the result is incomplete
        return {
            "messages": state.get("messages", []),
            "results": result.get("output", "No output from agent")
        }
    except Exception as e:
        print(f"Error in agent_node: {e}")
        return {
            "messages": state.get("messages", []),
            "results": f"Error processing request: {str(e)}"
        }

In [8]:
# Create the graph
workflow = StateGraph(AgentState)
workflow.add_node("agent", agent_node)
workflow.set_entry_point("agent")
workflow.set_finish_point("agent")
graph = workflow.compile()

# Run the graph
def run_graph(query: str):
    """Run the agent graph with the given query"""
    inputs = {
        "human_input": query,
        "messages": [],
        "results": {}
    }
    try:
        result = graph.invoke(inputs)
        
        # Check if the result contains the final JSON output
        if isinstance(result, dict) and "disease" in result["results"] and "description" in result["results"] and "icd_codes" in result["results"]:
            print("Final result obtained. Stopping execution.")
            return result["results"]
        
        return result["results"]
    except Exception as e:
        print(f"Error running graph: {e}")
        return f"Error: {str(e)}"


In [9]:
# Generate output in JSON and CSV formats
def generate_output(query: str, result: str):
    """Convert the agent's output to JSON and CSV formats"""
    try:
        # Try to parse the result as JSON first
        try:
            # See if the result is already a valid JSON
            output = json.loads(result) if isinstance(result, str) else result
        except (json.JSONDecodeError, TypeError):
            # If not a valid JSON, try to extract JSON from text
            # Look for JSON-like structure in the text
            try:
                # Try to find JSON-like structure in text
                start_idx = result.find('{')
                end_idx = result.rfind('}')
                if start_idx >= 0 and end_idx > start_idx:
                    json_str = result[start_idx:end_idx+1]
                    output = json.loads(json_str)
                else:
                    # Fallback: create structured data from unstructured output
                    output = {
                        "disease": query,
                        "description": result.split('\n')[0] if '\n' in result else result[:100],
                        "icd_codes": []
                    }
            except Exception:
                # Last resort fallback
                output = {
                    "disease": query,
                    "description": "Failed to parse agent output",
                    "icd_codes": []
                }
        
        # Extract relevant fields with proper error handling
        disease = output.get("disease", query)
        description = output.get("description", "No description available")
        icd_codes = output.get("icd_codes", [])
        if isinstance(icd_codes, str):
            icd_codes = [icd_codes]  # Convert string to list if needed
        icd_codes_str = ", ".join(icd_codes) if icd_codes else "No ICD codes found"
        
        # Generate JSON output
        json_output = {
            "disease": disease,
            "description": description,
            "icd_codes": icd_codes
        }
        
        # Generate CSV output
        csv_output = [["Disease", "Description", "ICD Codes"]]
        csv_output.append([disease, description, icd_codes_str])
        
        return json_output, csv_output
    except Exception as e:
        print(f"Error generating output: {e}")
        # Provide a fallback output
        json_output = {"error": str(e), "query": query}
        csv_output = [["Error", "Query"], [str(e), query]]
        return json_output, csv_output

In [10]:
# Main function
def main(query: str):
    """Main function to run the agent and save results"""
    print(f"Processing query: {query}")
    result = run_graph(query)
    print(f"Result from graph: {result}")
    
    json_output, csv_output = generate_output(query, result)
    
    # Save JSON output
    with open('output.json', 'w') as f:
        json.dump(json_output, f, indent=2)
    
    # Save CSV output
    with open('output.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(csv_output)
    
    print("JSON output:", json_output)
    print("CSV output has been saved to output.csv")
    return json_output

# Example usage
if __name__ == "__main__":
    main("Malaria")

Processing query: Malaria


> Entering new AgentExecutor chain...
Thought: I will first use VectorSearch to get a description of Malaria. Then, I will use TavilySearch to find the associated ICD codes.  Finally, I will combine this information into a JSON object as specified.

Action: VectorSearch

Action Input: Malaria


Thought: Now that I have the description from VectorSearch, I will use TavilySearch to find the ICD codes associated with Malaria.

Action: tavily_search_results_json

Action Input: "ICD codes for Malaria"

Thought: I will now combine the description retrieved from VectorSearch and the ICD codes from TavilySearch into the required JSON format.  I will assume VectorSearch returned a description and TavilySearch returned a list of ICD codes.  Since I don't have access to the actual outputs of these tools, I will use placeholder data.  In a real-world scenario, I would replace this placeholder data with the actual outputs from the tools.


Action: None (Result compilatio